Website:
https://jesparza17.github.io/Jesparza17/



# Spin to Win? An Analysis of Which Pitchers Were Hurt the Most After MLB's Crackdown on Sticky Stuff
### By: Julian Esparza and Sam Broth

### Background

The two of us are working together on our final tutorial project. We would like to analyze data about baseball, specifically about pitchers, to attempt to conclude which pitchers may have been using illegal “sticky stuff” prior to the crackdown on it by Major League Baseball. Three rules in the [MLB’s very detailed rulebook](https://img.mlbstatic.com/mlb-images/image/upload/mlb/atcjzj9j7wrgvsm8wnjq.pdf "2021 MLB Rulebook") essentially prohibit the application of “foreign substances” directly on the baseball. While these rules have been there for a long time, they were very rarely enforced. One anonymous former pitcher estimated that prior to June 2021, [an overwhelming majority of pitchers in the MLB used some substance(s)](https://www.si.com/mlb/2021/06/04/sticky-stuff-is-the-new-steroids-daily-cover "'This Should Be the Biggest Scandal in Sports'") to help them alter the movement and spin on their pitches. These substances range anywhere from sunscreen to glue. It always seemed that everyone accepted how the MLB and its umpires turned a blind eye to this - that is, until a few months ago. On June 21, 2021, commissioner Rob Manfred and Major League Baseball finally [decided to enforce the rules](https://www.espn.com/mlb/story/_/id/31682815/major-league-baseball-crackdown-sticky-substances-begins-regular-checks-pitchers-monday "Major League Baseball's crackdown on sticky substances begins with regular checks on pitchers Monday") and crackdown on sticky stuff. After all, the league-wide batting average was the [lowest it had been since 1968](https://www.espn.com/mlb/story/_/id/31371552/mlb-hopes-offensive-numbers-perk-historically-rough-april-hitters "MLB hopes offensive numbers perk up after historically rough April for hitters")! 
Using data from multiple sources, we hope to see which pitchers were negatively affected from this change. The dataset we are most excited about working with is from [Baseball Savant](https://baseballsavant.mlb.com/ "Baseball Savant"), a really great website for all things [sabermetrics](https://en.wikipedia.org/wiki/Sabermetrics "Sabermetrics"). There is a plethora of statistics we can use, and we are happy to explore all of them. A few that will really come in handy - average spin and average break by pitch. 



### Questions We Would Like to Answer
* Which pitchers’ average spin rate dropped the most after the enforcement of the rule? 

* Which pitchers’ expected batting average got higher after the change?

* Did some pitchers stop throwing a favorite pitch of theirs because they were better at it before the change?

* Is velocity affected by change in spin rate?


While we can most likely get most of our answers from the amazing datasets on Baseball Savant, [FanGraphs](https://www.fangraphs.com/ "FanGraphs") may also come in handy with more historical data. 

### Collaboration Plan
Our collaboration plan revolves around our shared GitHub repository, which will be where everything is easily accessible. We meet, mostly on Zoom, as often as required, and perhaps use VSCode if we need to be coding at the same time. We have both found that its LiveShare feature works very well for that.

### Data ETL

#### Extraction

In [20]:
##Importing Necessary Packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

The datasets shown below were extracted from [Baseball Savant](https://baseballsavant.mlb.com/leaderboard/active-spin "Active Spin Leaderboard"). In this website we are given many datasets which can be filtered based upon several different options in a drop down menu. This menu allows for data to be broken down by a certain statistic across players in the MLB.

## Load and Transform

In [21]:
#Loading first data frame from csv of actice spin of pitchers in 2021
spin_2020 = pd.read_csv("active-spin 2020.csv")

#Getting rid of rows we will not be using(what hand pithcer uses)
spin_2020 = spin_2020.drop(spin_2020.columns[2], axis = 1)

#Creating new column combining first and last name for ease
spin_2020['Name'] = spin_2020[' first_name'] + ' ' + spin_2020['last_name']

#Getting rid of previous name columns
spin_2020 = spin_2020.drop(spin_2020.columns[:2], axis = 1)

#Moving new name column to first column
position = spin_2020.pop("Name")
spin_2020.insert(0,'Name',position)
spin_2020.head()

,Name,active_spin_fourseam,active_spin_sinker,active_spin_cutter,active_spin_changeup,active_spin_fastball,active_spin_slider,active_spin_curve
0,Tejay Antone,NaN,85.4,NaN,98.9,NaN,41.0,81.0
1,Alex Cobb,NaN,97.3,NaN,NaN,NaN,NaN,65.7
2,Brady Singer,NaN,87.2,NaN,97.1,NaN,34.7,NaN
3,Ryan Weber,NaN,93.2,43.4,96.2,NaN,NaN,82.6
4,Ryan Yarbrough,NaN,97.6,43.1,94.8,NaN,NaN,78.3


In [22]:

break_2020 = pd.read_csv("pitch_movement 2020.csv")
break_2020 = break_2020.drop(break_2020.columns[3:9], axis =1)
break_2020 = break_2020.drop(break_2020.columns[3:8], axis =1)
#Creating new column combining first and last name for ease
break_2020['Name'] = break_2020[' first_name'] + ' ' + break_2020['last_name']

#Getting rid of previous name columns
break_2020 = break_2020.drop(break_2020.columns[1:3], axis = 1)

#Moving new name column to first column
position = break_2020.pop("Name")
break_2020.insert(0,'Name',position)
break_2020.head()


,Name,year,pitcher_break_z,league_break_z,diff_z,rise,pitcher_break_x,league_break_x,diff_x,tail,percent_rank_diff_z,percent_rank_diff_x
0,Lance Lynn,2020,15.4,-15.2,-0.2,-1,6.7,-7.6,-0.9,-12,0.487745,0.323529
1,Gerrit Cole,2020,11.5,-13.7,2.1,16,12.2,-8.0,4.2,53,0.906863,0.924020
2,Antonio Senzatela,2020,20.0,-14.9,-5.1,-34,5.9,-7.4,-1.5,-21,0.034314,0.245098
3,Andrew Heaney,2020,17.1,-17.1,-0.1,0,13.8,-8.6,5.2,60,0.509804,0.960784
4,Lucas Giolito,2020,12.5,-13.9,1.4,10,7.1,-6.5,0.6,8,0.794118,0.556373


In [23]:
break_2021 = pd.read_csv("pitch_movement 2021.csv")
break_2021 = break_2021.drop(break_2021.columns[3:9], axis =1)
break_2021 = break_2021.drop(break_2021.columns[3:8], axis =1)
#Creating new column combining first and last name for ease
break_2021['Name'] = break_2021[' first_name'] + ' ' + break_2021['last_name']

#Getting rid of previous name columns
break_2021 = break_2021.drop(break_2021.columns[1:3], axis = 1)

#Moving new name column to first column
position = break_2021.pop("Name")
break_2021.insert(0,'Name',position)
break_2021.head()



,Name,year,pitcher_break_z,league_break_z,diff_z,rise,pitcher_break_x,league_break_x,diff_x,tail,percent_rank_diff_z,percent_rank_diff_x
0,Robbie Ray,2021,13.5,-14.1,0.6,5,8.2,-7.1,1.1,15,0.650831,0.612827
1,Sean Manaea,2021,20.1,-21.8,-1.8,-8,13.8,-14.4,-0.6,-4,0.278689,0.327869
2,Tyler Mahle,2021,13.9,-15.1,1.2,8,12.1,-7.7,4.4,57,0.776722,0.923990
3,Kevin Gausman,2021,14.2,-14.2,0.0,0,10.4,-7.1,3.3,47,0.527316,0.845606
4,J.A. Happ,2021,16.1,-16.0,0.0,0,7.9,-6.7,1.2,18,0.527316,0.624703


In [24]:
#Merged Spin rate of 2020 and break of 2020 for a data set that contained all necessary info for 2020
year2020 = spin_2020.merge(break_2020, on = ['Name'], how = 'outer')
year2020.head()

,Name,active_spin_fourseam,active_spin_sinker,active_spin_cutter,active_spin_changeup,active_spin_fastball,active_spin_slider,active_spin_curve,year,pitcher_break_z,league_break_z,diff_z,rise,pitcher_break_x,league_break_x,diff_x,tail,percent_rank_diff_z,percent_rank_diff_x
0,Tejay Antone,NaN,85.4,NaN,98.9,NaN,41.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alex Cobb,NaN,97.3,NaN,NaN,NaN,NaN,65.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brady Singer,NaN,87.2,NaN,97.1,NaN,34.7,NaN,2020.0,19.2,-21.5,-2.3,-11.0,13.7,-14.6,-0.9,-6.0,0.241245,0.284047
3,Ryan Weber,NaN,93.2,43.4,96.2,NaN,NaN,82.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ryan Yarbrough,NaN,97.6,43.1,94.8,NaN,NaN,78.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#Merged Spin rate of 2021 and break of 2021 for a data set that contained all necessary info for 2021
year2021 = spin_2021.merge(break_2021, on = ['Name'], how = 'outer')
year2021.head()

NameError: name 'spin_2021' is not defined

In [46]:
fb2020 = pd.read_csv("fastball_stuff2020.csv")
fb2020['Name'] = fb2020[' first_name'] + ' ' + fb2020['last_name']
fb2020.drop(["player_id", "Unnamed: 7", "last_name", " first_name", "year"], axis=1, inplace=True)

position = fb2020.pop("Name")
fb2020.insert(0,'Name',position)
fb2020.rename(columns={"n_fastball_formatted": "Fastball %"}, inplace=True)

fb2020.sort_values("fastball_avg_spin", inplace=True, ascending=False)

fb2020.head()

,Name,Fastball %,fastball_avg_speed,fastball_avg_spin
117,Trevor Bauer,67.5,91.0,2817
445,Corbin Burnes,67.0,94.7,2793
259,Lucas Sims,48.1,93.9,2748
157,Jason Adam,53.5,94.8,2722
63,Yu Darvish,63.8,90.1,2698


In [47]:
fb2021 = pd.read_csv("fastball_stuff2021.csv")
fb2021['Name'] = fb2021[' first_name'] + ' ' + fb2021['last_name']
fb2021.drop(["player_id", "Unnamed: 7", "last_name", " first_name", "year"], axis=1, inplace=True)

position = fb2021.pop("Name")
fb2021.insert(0,'Name',position)
fb2021.rename(columns={"n_fastball_formatted": "Fastball %"}, inplace=True)

fb2021.sort_values("fastball_avg_spin", inplace=True, ascending=False)

fb2021.head()

,Name,Fastball %,fastball_avg_speed,fastball_avg_spin
126,Trevor Bauer,67.6,91.2,2792
600,Corbin Burnes,62.9,95.5,2734
20,Daniel Bard,48.5,97.4,2687
350,Tejay Antone,32.1,96.8,2677
232,Dillon Maples,51.7,95.2,2641


In [79]:
fbboth = fb2020.merge(fb2021, on="Name", how="inner", suffixes=('_2020','_2021'))
fbboth["Fastball Spin Diff (%)"] = ((fbboth["fastball_avg_spin_2021"] - fbboth["fastball_avg_spin_2020"]) / (fbboth["fastball_avg_spin_2020"])) * 100
fbboth.sort_values("Fastball Spin Diff (%)", inplace=True)
fbboth.set_index("Name", inplace=True)
fbboth.head()

,Fastball %_2020,fastball_avg_speed_2020,fastball_avg_spin_2020,Fastball %_2021,fastball_avg_speed_2021,fastball_avg_spin_2021,Fastball Spin Diff (%)
Name,,,,,,,
Brett Anderson,54.5,89.2,2015,53.8,88.6,1759,-12.704715
Tarik Skubal,60.1,94.4,2422,56.0,94.4,2152,-11.147812
Taylor Widener,65.1,94.5,2396,66.5,92.6,2143,-10.559265
Junior Guerra,60.6,94.0,2446,56.0,92.8,2199,-10.098119
Cody Ponce,68.3,91.9,2529,61.0,92.0,2278,-9.924871


In [80]:
fbboth["Fastball Spin Diff (%)"].mean()

-1.1568592946162721

In [87]:
# did pitchers stop using fastballs
bigdrop = fbboth[fbboth["Fastball Spin Diff (%)"] <= -5]
bigdrop["Fastball Use Diff (%)"] = (bigdrop["Fastball %_2021"] - bigdrop["Fastball %_2020"])

bigdrop.head()   # There are 38 pitchers here

<ipython-input-87-20a1fe8a1650>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigdrop["Fastball Use Diff (%)"] = (bigdrop["Fastball %_2021"] - bigdrop["Fastball %_2020"])


,Fastball %_2020,fastball_avg_speed_2020,fastball_avg_spin_2020,Fastball %_2021,fastball_avg_speed_2021,fastball_avg_spin_2021,Fastball Spin Diff (%),Fastball Use Diff (%)
Name,,,,,,,,
Brett Anderson,54.5,89.2,2015,53.8,88.6,1759,-12.704715,-0.7
Tarik Skubal,60.1,94.4,2422,56.0,94.4,2152,-11.147812,-4.1
Taylor Widener,65.1,94.5,2396,66.5,92.6,2143,-10.559265,1.4
Junior Guerra,60.6,94.0,2446,56.0,92.8,2199,-10.098119,-4.6
Cody Ponce,68.3,91.9,2529,61.0,92.0,2278,-9.924871,-7.3


In [88]:
bigdrop["Fastball Use Diff (%)"].mean()

-0.11052631578947357